# Step 1: Connect to Qdrant

In [ ]:
import minsearch
import json
from openai import OpenAI  # OpenAI API client
import pinecone  # Pinecone client library
# print(pinecone.__version__)  # Print Pinecone version (optional)
from pinecone import Pinecone, ServerlessSpec  # For setup and serverless config
import hashlib  # For hashing (e.g., file IDs)
import os  # OS operations and env vars
from datetime import datetime  # Date and time handling
from qdrant_client import QdrantClient
from tqdm import tqdm  # Progress bars (e.g., looping through files)
import requests  # HTTP requests (e.g., for web scraping or API calls)
import re  # Regular expressions (e.g., for pattern matching in strings)
from bs4 import BeautifulSoup  # HTML parsing
from dotenv import load_dotenv  # Load .env file

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='llm-sparse-and-dense'), CollectionDescription(name='llm2-sparse'), CollectionDescription(name='hm2-q3q4'), CollectionDescription(name='homework2'), CollectionDescription(name='llm-rag')])

In [70]:
from qdrant_client import QdrantClient

model_handle = "jinaai/jina-embeddings-v2-small-en"


In [71]:
from qdrant_client.http import models
from fastembed import TextEmbedding
model_handle = "jinaai/jina-embeddings-v2-small-en"

query = "I just discovered the course. Can I join now?"

points = []
id = 0
point = models.PointStruct(
            id=id,
            vector=models.Document(text=query, model=model_handle),
            payload={
                "query": query,
            } 
        )
points.append(point)


id += 1

In [72]:
embed_model = TextEmbedding(model_name=model_handle)
embedding_vector = list(embed_model.embed(query))
embedding1 = embedding_vector[0]

In [44]:
min_value = min(embedding1)
print("Minimal value:", min_value)

Minimal value: -0.11726373885183883


In [45]:
import numpy as np

In [73]:
doc = 'Can I still join the course after the start date?'
embed_model = TextEmbedding(model_name=model_handle)
embedding_vector2 = list(embed_model.embed(doc))
embedding2 = embedding_vector2[0]

In [74]:
embedding2.dot(embedding1)

np.float64(0.9008528895674548)

In [77]:
import json
with open('q3q4hm2.json', 'rt') as rk_in:
    docs_raw = json.load(rk_in)
docs_raw

[{'documents': [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
    'section': 'General course-related questions',
    'question': 'Course - Can I still join the course after the start date?',
    'course': 'data-engineering-zoomcamp'},
   {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
    'section': 'General course-related questions',
    'question': 'Course - Can I follow the course after it finishes?',
    'course': 'data-engineering-zoomcamp'},
   {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of 

In [ ]:
# Define the collection name
collection_name = "hm2-q3q4"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [79]:
points = []
id = 0

for course in docs_raw:
    for doc in course['documents']:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": doc['course'],
                "question": doc['question']
            }
        )
        points.append(point)
        id += 1


In [80]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [81]:
query = "I just discovered the course. Can I join now?"

def search(query, limit=4):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [82]:
print(search(query, limit=4))

points=[ScoredPoint(id=1, version=1, score=0.8182378, payload={'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp', 'question': 'Course - Can I follow the course after it finishes?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=1, score=0.80853975, payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel w

In [83]:
embed_model = TextEmbedding(model_name=model_handle)
embedding_vector1 = list(embed_model.embed(doc))
embedding1 = embedding_vector1[0]


query = "I just discovered the course. Can I join now?"
embedding_vector2 = list(embed_model.embed(query))
embedding2 = embedding_vector2[0]


embedding2.dot(embedding1)

np.float64(0.7371250511873899)

In [ ]:
# Define the collection name
collection_name = "hm2-q3q4-2"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `hm2-q3q4-2` already exists!"},"time":0.000177556}'

In [95]:
embed_model = TextEmbedding(model_name=model_handle)

points = []
id = 0

for course in docs_raw:
    for doc in course['documents']:
        fulltext = doc['question'] + " " + doc['text']
        embedding = list(embed_model.embed([fulltext]))[0]
        point = models.PointStruct(
            id=id,
            vector=embedding,
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": doc['course'],
                "question": doc['question']
            }
        )
        points.append(point)
        id += 1


In [96]:
client.upsert(
    collection_name=collection_name,
    points=points
)


query = "I just discovered the course. Can I join now?"

def search(query, limit=4):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [97]:
print(search(query, limit=4))

points=[ScoredPoint(id=0, version=2, score=0.8514543, payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp', 'question': 'Course - Can I still join the course after the start date?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=2, score=0.8436594, payload={'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp', 'question': 'Course - Can I follow the course after it finishes?'},

In [102]:
import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [108]:
model_handle = "BAAI/bge-small-en"

In [112]:
# Define the collection name
collection_name = "hm2-q8"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [113]:
embed_model = TextEmbedding(model_name=model_handle)

points = []
id = 0

for course in docs_raw:
    for doc in course['documents']:
        fulltext = doc['question'] + " " + doc['text']
        embedding = list(embed_model.embed([fulltext]))[0]
        point = models.PointStruct(
            id=id,
            vector=embedding,
            payload={
                "text": doc['text'],
                "section": doc['section']
            }
        )
        points.append(point)
        id += 1


In [114]:
embed_model = TextEmbedding(model_name=model_handle)
embedding_vector1 = list(embed_model.embed(doc))
embedding1 = embedding_vector1[0]


query = "I just discovered the course. Can I join now?"
embedding_vector2 = list(embed_model.embed(query))
embedding2 = embedding_vector2[0]


embedding2.dot(embedding1)

np.float32(0.78359246)